In [1]:
import numpy as np
import pandas as pd
import plotly.express as px

In [2]:
# INPUTS
threshold1 = 1
threshold2 = threshold1-0.1
max_tim = 500
leverage = 1
country = {
    'us': 'SPY',
    'france': 'EWQ',
    'germany': 'EWG',
    'uk': 'EWU',
    #'japan': 'EWJ',
    'italy': 'EWI',
    'swiss': 'EWL',
}
bond = pd.read_csv(f'data/IGSB.csv', index_col=0, parse_dates=True)['price']
bond_change = bond.pct_change()

In [3]:
index_dict = dict()
ptf_results = dict()
for name, ticker in country.items():
    price = pd.read_csv(f'data/{ticker}.csv', index_col=0, parse_dates=True)['price']
    index_dict[name] = price
    df = pd.DataFrame()
    df['price'] = price
    df['return'] = price.pct_change()
    df['log_return'] = np.log(price).diff()
    df['rolling_std'] = df['log_return'].rolling(10).std() * np.sqrt(252)
    df['dd'] = df['price'] / df['price'].expanding().max()
    df = df.dropna()
    df['fwd_return'] = df['price'].shift(-252)/df['price'] - 1
    #df = df.dropna()
    #px.scatter(df.set_index('dd')['fwd_return'])
    tim = 0
    ptf = dict()
    units = dict()
    activated = False
    init = False
    for n, index in enumerate(df.index, start=1):
        
        yesterday_index = df.index[n-2]
        today_index = df.index[n-1]

        if init == False:
            init = True
            units[yesterday_index] = 0
            ptf[yesterday_index] = 100_000

        if units[yesterday_index] == 0:
            #ptf[today_index] = ptf[yesterday_index] * (1 + bond_change.loc[today_index])
            ptf[today_index] = ptf[yesterday_index] * (1 + 0.036/252)
        else:
            ptf[today_index] = ptf[yesterday_index] + units[yesterday_index] * (df.loc[today_index, 'price'] - df.loc[yesterday_index, 'price'])

        if df.loc[index, 'rolling_std'] > threshold1 and activated == False:
            activated = True
            units[today_index] = 0

        elif units[yesterday_index] != 0 and df.loc[index, 'dd'] >= 1:
            units[today_index] = 0

        elif df.loc[index, 'rolling_std'] < threshold2 and activated == True:
            tim = 0
            activated = False
            units[today_index] = (ptf[today_index] / df.loc[index, 'price']) * leverage
        
        else:
            units[today_index] = units[yesterday_index]

        tim += 1
    ptf = pd.Series(ptf).sort_index()
    ptf_results[name] = ptf

In [4]:
index_df = pd.DataFrame(index_dict)
-(((index_df.diff() > 0) * index_df.diff()).sum()) / (((index_df.diff() < 0) * index_df.diff()).sum())

us         1.141434
france     1.041982
germany    1.048433
uk         1.037830
italy      1.022710
swiss      1.076848
dtype: float64

In [5]:
ptf_results_df = pd.DataFrame(ptf_results)
np.exp(np.log(ptf_results_df.iloc[-1] / ptf_results_df.iloc[0])/17) - 1

us         0.084936
france     0.093854
germany    0.111991
uk         0.094202
italy      0.050428
swiss      0.078648
dtype: float64

In [6]:
px.line(pd.DataFrame(ptf_results))

In [17]:
dd = (index_df / index_df.expanding().max()) * 100
px.line(dd-100)

In [18]:
np.exp(np.log(index_df.iloc[-1] / index_df.iloc[0])/17) - 1

us         0.115918
france     0.044823
germany    0.053596
uk         0.035218
italy      0.020588
swiss      0.072061
dtype: float64